# 자동 벡터화

## 수동으로 벡터화 하는법

아래 코드는 2개의 1차원 벡터끼리 컨볼루션(합성곱) 연산을 하는 코드입니다:

In [3]:
import jax
import jax.numpy as jnp

x = jnp.arange(5)
w = jnp.array([2., 3., 4.])

def convolve(x, w):
    output = []
    for i in range(1, len(x) - 1):
        output.append(jnp.dot(x[i-1:i+2], w))
    return jnp.array(output)

convolve(x, w)

Array([11., 20., 29.], dtype=float32)

만약 이러한 연산을 여러개의 w와 여러개의 x에 대해서 진행하고 싶다고 가정합시다:

In [4]:
# 2개를 쌓았습니다.
xs = jnp.stack([x, x])
ws = jnp.stack([w, w])

In [6]:
def manually_batched_convolve(xs, ws):
    output = []
    for i in range(xs.shape[0]):
        output.append(convolve(xs[i], ws[i]))
    return jnp.stack(output)

manually_batched_convolve(xs, ws)

Array([[11., 20., 29.],
       [11., 20., 29.]], dtype=float32)

올바른 값을 내주지만, 효율적인 방법은 아닙니다.

위 계산을 효율적으로 하기 위해서는 벡터 연산이 이루어질 수 있도록 함수를 재작성해야 합니다(이렇게 같은 여러 연산을 한번에 처리하는 방식을 batch라고 부릅니다). 이는 구현의 난이도가 높지는 않지만, 함수가 input 값을 다루는 방법을 바꿔주어야 합니다. 예를들어 여러 벡터에 대해서 행렬곱을 하게 되면 인풋이 자연스럽게 1차원에서 2차원으로 바뀌게 되고, 이에 따라 함수 내부에서 처리하는 배열의 축(axis)도 바뀌게 되어, 단일 연산과 복수 연산을 판단하는 로직이 함수에 또 들어가게 되는 식입니다.

In [7]:
def manually_vectorized_convolve(xs, ws):
  output = []
  for i in range(1, xs.shape[-1] -1):
    output.append(jnp.sum(xs[:, i-1:i+2] * ws, axis=1))
  return jnp.stack(output, axis=1)

manually_vectorized_convolve(xs, ws)

Array([[11., 20., 29.],
       [11., 20., 29.]], dtype=float32)

이러한 재구현은 지저분하고 함수의 복잡도를 높여 에러를 유발할 수 있습니다. Jax는 연산을 벡터화시킬 수 있는 더 우아한 방법이 존재합니다.

## 자동 벡터화

`jax.vmap()`은 우리가 만든 함수를 벡터화되는 함수로 자동으로 변환해줍니다:

In [9]:
auto_batch_convolve = jax.vmap(convolve)

auto_batch_convolve(xs, ws)

Array([[11., 20., 29.],
       [11., 20., 29.]], dtype=float32)

`jax.vmap()`은 `jax.jit()`과 비슷한 방식으로 함수를 tracing하여 jax가 이해할 수 있는 jaxpr로 표현하게 됩니다. 그리고 해당 표현해 배치 처리가 이뤄질 축(axis)를 추가합니다.

만약 배치 처리가 이뤄져야 할 축이 배열의 0번째 축이 아니라면, `in_axes`와 `out_axis` 인자를 통해 원하는 축을 지정할 수 있습니다. 

In [12]:
auto_batch_convolve_v2 = jax.vmap(convolve, in_axes=1, out_axes=0)
auto_batch_convolve_v3 = jax.vmap(convolve, in_axes=1, out_axes=1)
xst = jnp.transpose(xs)
wst = jnp.transpose(ws)

print(auto_batch_convolve_v2(xst, wst))
print(auto_batch_convolve_v3(xst, wst))

[[11. 20. 29.]
 [11. 20. 29.]]
[[11. 11.]
 [20. 20.]
 [29. 29.]]


`jax.vmap()`은 함수에 들어오는 어떤 인자를 배치연산할 것인지도 지정해줄 수 있습니다. 예를들어 여러개의 x를 하나의 w에 대해서 연산하고 싶다면, `w`에 해당하는 위치에 아래와 같이 `None`을 지정해주면 됩니다:

In [14]:
batch_convolve_v4 = jax.vmap(convolve, in_axes=[0, None])

batch_convolve_v4(xs, w)

Array([[11., 20., 29.],
       [11., 20., 29.]], dtype=float32)

## 변환함수 같이쓰기

`jax.vmap()` 위에 `jax.jit()` 를 씌우던, `jax.jit()` 위에 `jax.vmap()` 를 씌우던 전혀 무관합니다.

In [15]:
jitted_batch_convolve = jax.jit(auto_batch_convolve)

jitted_batch_convolve(xs, ws)

Array([[11., 20., 29.],
       [11., 20., 29.]], dtype=float32)